In [1]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

from sklearn.datasets import load_digits
digits = load_digits()

In [2]:
from random import randint

for i in range(64):
    ax = plt.subplot(8, 8, i+1)
    ax.axis('off')
    plt.imshow(digits.images[randint(0, digits.images.shape[0])], cmap='Greys')
plt.show()

In [3]:
X = digits.images.reshape(digits.images.shape[0], digits.images.shape[1] * digits.images.shape[2])[:1617]
y = digits.target[:1617]

In [4]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [5]:
def get_winner(class1):
    
    numbers = [*range(10)] # number 2 > number 1
    number_of_classification = len(numbers)
    #number_of_sampling = 100

    list_y = y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

        
    class2 = []

    class1_indexes = []
    class2_indexes = []

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("number of class1:{}".format(len(class1_label)))
    print("number of class2:{}".format(len(class2_label)))

    samplesize = 1617
    x_inputs =[tuple(c) for c in X[class1_indexes].tolist()] + [tuple(c) for c in X[class2_indexes].tolist()]
    x_outputs = [tuple([c]) for c in class1_label + class2_label]
    
    
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            outputs = []
            for xi in x_inputs:
                output = net.activate(xi)
                outputs.append(output)


            px_outputs = softmax(np.array(outputs).reshape(samplesize, 2), axis=1)
            # the index of maximum in each line
            pred_outputs = np.argmax(px_outputs, axis = 1)
            real_outputs = np.array(x_outputs).reshape(samplesize,)

            acc = np.sum(pred_outputs == real_outputs)/samplesize

            genome.fitness = acc

    def run(config_file):
        # Load configuration.
        config = neat.Config(
            neat.DefaultGenome,
            neat.DefaultReproduction,
            neat.DefaultSpeciesSet,
            neat.DefaultStagnation,
            config_file,
        )

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)

        # add a stdout reporter to show progress in the terminal
        reporter = neat.StdOutReporter(False)
        p.add_reporter(reporter)
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        #checkpointer = neat.Checkpointer(100)
        #p.add_reporter(checkpointer)
        # Run for up to 300 generations.
        winner = p.run(eval_genomes, 6)

        return [stats, winner]
    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward-mnist2")
    [stats, winner] = run(config_path)
    
    return winner

In [6]:
import numpy as np
number_codes = []
for i in range(2**(10-1), 2**10-1):
    for j in bin(i).split('b')[1]:
        number_codes.append(int(j))
number_codes = np.array(number_codes).reshape(2**9 - 1,10).T.tolist() 

In [7]:
classes = []
for i in range(2**(10-1) - 1):
    code_list = np.array(number_codes).T[i]
    classes.append(np.where(code_list==1)[0].tolist())

In [8]:
winner_list = []

for i,class1 in enumerate(classes):
    print("running:{}/511".format(i))
    winner = get_winner(class1)
    winner_list.append(winner)

running:0/511
number of class1:162
number of class2:1455

 ****** Running generation 0 ****** 

Population's average fitness: 0.60066 stdev: 0.27126
Best fitness: 0.97712 - size: (2, 13) - species 3 - id 173
Average adjusted fitness: 0.504
Mean genetic distance 2.892, standard deviation 0.482
Population of 200 members in 24 species
Total extinctions: 0
Generation time: 6.489 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.74772 stdev: 0.24106
Best fitness: 0.97712 - size: (2, 13) - species 3 - id 173
Average adjusted fitness: 0.633
Mean genetic distance 2.990, standard deviation 0.473
Population of 202 members in 29 species
Total extinctions: 0
Generation time: 6.023 sec (6.256 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.78600 stdev: 0.21645
Best fitness: 0.97712 - size: (2, 13) - species 24 - id 173
Average adjusted fitness: 0.666
Mean genetic distance 2.993, standard deviation 0.502
Population of 201 members in 30 species

Mean genetic distance 3.183, standard deviation 0.470
Population of 197 members in 38 species
Total extinctions: 0
Generation time: 12.338 sec (11.594 average)
running:4/511
number of class1:322
number of class2:1295

 ****** Running generation 0 ****** 

Population's average fitness: 0.54994 stdev: 0.21310
Best fitness: 0.85096 - size: (2, 13) - species 1 - id 2
Average adjusted fitness: 0.428
Mean genetic distance 2.922, standard deviation 0.482
Population of 200 members in 23 species
Total extinctions: 0
Generation time: 10.120 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.66418 stdev: 0.19017
Best fitness: 0.86333 - size: (3, 14) - species 4 - id 294
Average adjusted fitness: 0.481
Mean genetic distance 3.024, standard deviation 0.486
Population of 200 members in 27 species
Total extinctions: 0
Generation time: 10.371 sec (10.246 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.69850 stdev: 0.17176
Best fitness: 0.88064 - 

Population's average fitness: 0.59671 stdev: 0.08473
Best fitness: 0.79530 - size: (4, 16) - species 29 - id 948
Average adjusted fitness: 0.205
Mean genetic distance 3.129, standard deviation 0.480
Population of 198 members in 38 species
Total extinctions: 0
Generation time: 8.685 sec (9.808 average)
running:8/511
number of class1:325
number of class2:1292

 ****** Running generation 0 ****** 

Population's average fitness: 0.54702 stdev: 0.21364
Best fitness: 0.85405 - size: (2, 13) - species 5 - id 47
Average adjusted fitness: 0.403
Mean genetic distance 2.902, standard deviation 0.469
Population of 200 members in 22 species
Total extinctions: 0
Generation time: 6.886 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.67861 stdev: 0.18523
Best fitness: 0.88064 - size: (3, 15) - species 14 - id 284
Average adjusted fitness: 0.511
Mean genetic distance 3.007, standard deviation 0.515
Population of 204 members in 27 species
Total extinctions: 0
Generation time: 

Population's average fitness: 0.60326 stdev: 0.08227
Best fitness: 0.76562 - size: (5, 19) - species 17 - id 746
Average adjusted fitness: 0.267
Mean genetic distance 3.082, standard deviation 0.520
Population of 197 members in 28 species
Total extinctions: 0
Generation time: 11.296 sec (10.619 average)
running:12/511
number of class1:485
number of class2:1132

 ****** Running generation 0 ****** 

Population's average fitness: 0.52330 stdev: 0.15559
Best fitness: 0.81818 - size: (2, 13) - species 3 - id 79
Average adjusted fitness: 0.239
Mean genetic distance 2.882, standard deviation 0.467
Population of 199 members in 27 species
Total extinctions: 0
Generation time: 8.720 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.58258 stdev: 0.14714
Best fitness: 0.81818 - size: (2, 13) - species 3 - id 79
Average adjusted fitness: 0.294
Mean genetic distance 3.001, standard deviation 0.468
Population of 199 members in 31 species
Total extinctions: 0
Generation time:

Population's average fitness: 0.56309 stdev: 0.06656
Best fitness: 0.69821 - size: (3, 13) - species 21 - id 808
Average adjusted fitness: 0.191
Mean genetic distance 2.989, standard deviation 0.500
Population of 201 members in 28 species
Total extinctions: 0
Generation time: 17.050 sec (16.467 average)
running:16/511
number of class1:326
number of class2:1291

 ****** Running generation 0 ****** 

Population's average fitness: 0.54507 stdev: 0.21042
Best fitness: 0.87755 - size: (2, 13) - species 2 - id 198
Average adjusted fitness: 0.387
Mean genetic distance 2.952, standard deviation 0.496
Population of 201 members in 26 species
Total extinctions: 0
Generation time: 13.834 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.67903 stdev: 0.17182
Best fitness: 0.88497 - size: (3, 15) - species 24 - id 373
Average adjusted fitness: 0.476
Mean genetic distance 2.959, standard deviation 0.480
Population of 201 members in 30 species
Total extinctions: 0
Generation t

...


 ****** Running generation 0 ****** 

Population's average fitness: 0.43403 stdev: 0.20675
Best fitness: 0.82560 - size: (2, 13) - species 2 - id 55
Average adjusted fitness: 0.295
Mean genetic distance 2.898, standard deviation 0.476
Population of 201 members in 20 species
Total extinctions: 0
Generation time: 24.393 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.56697 stdev: 0.20331
Best fitness: 0.82560 - size: (2, 13) - species 2 - id 55
Average adjusted fitness: 0.407
Mean genetic distance 2.971, standard deviation 0.496
Population of 204 members in 26 species
Total extinctions: 0
Generation time: 25.263 sec (24.828 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.66256 stdev: 0.17938
Best fitness: 0.82560 - size: (2, 13) - species 2 - id 55
Average adjusted fitness: 0.453
Mean genetic distance 2.995, standard deviation 0.492
Population of 202 members in 26 species
Total extinctions: 0
Generation time: 24.852 sec (24.836 


 ****** Running generation 0 ****** 

Population's average fitness: 0.45271 stdev: 0.14044
Best fitness: 0.74397 - size: (2, 13) - species 2 - id 101
Average adjusted fitness: 0.196
Mean genetic distance 2.842, standard deviation 0.391
Population of 199 members in 32 species
Total extinctions: 0
Generation time: 25.476 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.55269 stdev: 0.13621
Best fitness: 0.74397 - size: (2, 13) - species 6 - id 101
Average adjusted fitness: 0.242
Mean genetic distance 2.957, standard deviation 0.429
Population of 200 members in 34 species
Total extinctions: 0
Generation time: 30.515 sec (27.995 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.60237 stdev: 0.13375
Best fitness: 0.74459 - size: (4, 16) - species 24 - id 527
Average adjusted fitness: 0.302
Mean genetic distance 2.995, standard deviation 0.450
Population of 200 members in 35 species
Total extinctions: 0
Generation time: 26.225 sec (27.


 ****** Running generation 0 ****** 

Population's average fitness: 0.44023 stdev: 0.21300
Best fitness: 0.80767 - size: (2, 13) - species 4 - id 99
Average adjusted fitness: 0.240
Mean genetic distance 2.892, standard deviation 0.470
Population of 199 members in 24 species
Total extinctions: 0
Generation time: 19.089 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.59008 stdev: 0.19746
Best fitness: 0.80767 - size: (3, 15) - species 12 - id 280
Average adjusted fitness: 0.410
Mean genetic distance 2.980, standard deviation 0.474
Population of 197 members in 29 species
Total extinctions: 0
Generation time: 23.238 sec (21.164 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.65992 stdev: 0.16526
Best fitness: 0.86456 - size: (3, 15) - species 19 - id 408
Average adjusted fitness: 0.430
Mean genetic distance 3.019, standard deviation 0.503
Population of 198 members in 30 species
Total extinctions: 0
Generation time: 23.421 sec (21.

Mean genetic distance 2.997, standard deviation 0.444
Population of 197 members in 32 species
Total extinctions: 0
Generation time: 21.662 sec (21.032 average)
running:506/511
number of class1:1295
number of class2:322

 ****** Running generation 0 ****** 

Population's average fitness: 0.43215 stdev: 0.21002
Best fitness: 0.80829 - size: (2, 13) - species 1 - id 117
Average adjusted fitness: 0.274
Mean genetic distance 2.897, standard deviation 0.479
Population of 200 members in 23 species
Total extinctions: 0
Generation time: 17.610 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.61573 stdev: 0.19462
Best fitness: 0.85591 - size: (3, 14) - species 5 - id 321
Average adjusted fitness: 0.418
Mean genetic distance 3.035, standard deviation 0.501
Population of 201 members in 30 species
Total extinctions: 0
Generation time: 19.199 sec (18.404 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.65232 stdev: 0.19270
Best fitness: 0.8559

Population's average fitness: 0.74435 stdev: 0.23997
Best fitness: 0.90353 - size: (4, 16) - species 39 - id 850
Average adjusted fitness: 0.586
Mean genetic distance 3.036, standard deviation 0.446
Population of 208 members in 47 species
Total extinctions: 0
Generation time: 21.671 sec (22.722 average)
running:510/511
number of class1:1455
number of class2:162

 ****** Running generation 0 ****** 

Population's average fitness: 0.44159 stdev: 0.27531
Best fitness: 0.89981 - size: (2, 13) - species 2 - id 73
Average adjusted fitness: 0.361
Mean genetic distance 2.875, standard deviation 0.459
Population of 200 members in 23 species
Total extinctions: 0
Generation time: 20.364 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.63746 stdev: 0.26691
Best fitness: 0.90043 - size: (3, 14) - species 1 - id 340
Average adjusted fitness: 0.566
Mean genetic distance 2.926, standard deviation 0.454
Population of 202 members in 28 species
Total extinctions: 0
Generation ti

In [9]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-mnist2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

In [10]:
winner_net0 = neat.nn.FeedForwardNetwork.create(winner_list[0], config)

In [11]:
def get_pred_real(i):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(10)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = digits.images.reshape(digits.images.shape[0], digits.images.shape[1] * digits.images.shape[2])[1617:]
    test_y = digits.target[1617:]

    list_y = test_y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("class1:{}/{}:class2".format(len(class1_label),len(class2_label)))

    testsamplesize = 180
    test_x_inputs =[tuple(c) for c in test_X[class1_indexes].tolist()] + [tuple(c) for c in test_X[class2_indexes].tolist()]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    acc = np.sum(pred_outputs == real_outputs)/testsamplesize
    print("Accuracy: {}".format(acc))
    
    return[pred_outputs, real_outputs]

In [12]:
pred = []
real = []
for i in range(len(classes)):
    [pred_outputs, real_outputs] = get_pred_real(i)
    pred.append(pred_outputs)
    real.append(real_outputs)

class1:16/164:class2
Accuracy: 0.9611111111111111
class1:34/146:class2
Accuracy: 0.95
class1:33/147:class2
Accuracy: 0.8833333333333333
class1:51/129:class2
Accuracy: 0.8
class1:35/145:class2
Accuracy: 0.8777777777777778
class1:53/127:class2
Accuracy: 0.8166666666666667
class1:52/128:class2
Accuracy: 0.7833333333333333
class1:70/110:class2
Accuracy: 0.8277777777777777
class1:34/146:class2
Accuracy: 0.9555555555555556
class1:52/128:class2
Accuracy: 0.8111111111111111
class1:51/129:class2
Accuracy: 0.7777777777777778
class1:69/111:class2
Accuracy: 0.8
class1:53/127:class2
Accuracy: 0.7666666666666667
class1:71/109:class2
Accuracy: 0.7444444444444445
class1:70/110:class2
Accuracy: 0.7166666666666667
class1:88/92:class2
Accuracy: 0.6833333333333333
class1:34/146:class2
Accuracy: 0.8944444444444445
class1:52/128:class2
Accuracy: 0.9111111111111111
class1:51/129:class2
Accuracy: 0.8111111111111111
class1:69/111:class2
Accuracy: 0.8166666666666667
class1:53/127:class2
Accuracy: 0.788888888888

Accuracy: 0.7277777777777777
class1:88/92:class2
Accuracy: 0.6944444444444444
class1:106/74:class2
Accuracy: 0.7277777777777777
class1:90/90:class2
Accuracy: 0.7111111111111111
class1:108/72:class2
Accuracy: 0.7166666666666667
class1:107/73:class2
Accuracy: 0.65
class1:125/55:class2
Accuracy: 0.7444444444444445
class1:89/91:class2
Accuracy: 0.8388888888888889
class1:107/73:class2
Accuracy: 0.7833333333333333
class1:106/74:class2
Accuracy: 0.7166666666666667
class1:124/56:class2
Accuracy: 0.7555555555555555
class1:108/72:class2
Accuracy: 0.75
class1:126/54:class2
Accuracy: 0.7111111111111111
class1:125/55:class2
Accuracy: 0.85
class1:143/37:class2
Accuracy: 0.8277777777777777
class1:51/129:class2
Accuracy: 0.7555555555555555
class1:69/111:class2
Accuracy: 0.7833333333333333
class1:68/112:class2
Accuracy: 0.7888888888888889
class1:86/94:class2
Accuracy: 0.8166666666666667
class1:70/110:class2
Accuracy: 0.7611111111111111
class1:88/92:class2
Accuracy: 0.7611111111111111
class1:87/93:class

class1:108/72:class2
Accuracy: 0.7944444444444444
class1:126/54:class2
Accuracy: 0.7833333333333333
class1:110/70:class2
Accuracy: 0.6888888888888889
class1:128/52:class2
Accuracy: 0.7611111111111111
class1:127/53:class2
Accuracy: 0.75
class1:145/35:class2
Accuracy: 0.8111111111111111
class1:91/89:class2
Accuracy: 0.6888888888888889
class1:109/71:class2
Accuracy: 0.6666666666666666
class1:108/72:class2
Accuracy: 0.8222222222222222
class1:126/54:class2
Accuracy: 0.7944444444444444
class1:110/70:class2
Accuracy: 0.7166666666666667
class1:128/52:class2
Accuracy: 0.8277777777777777
class1:127/53:class2
Accuracy: 0.8055555555555556
class1:145/35:class2
Accuracy: 0.9
class1:109/71:class2
Accuracy: 0.7833333333333333
class1:127/53:class2
Accuracy: 0.8222222222222222
class1:126/54:class2
Accuracy: 0.7833333333333333
class1:144/36:class2
Accuracy: 0.8722222222222222
class1:128/52:class2
Accuracy: 0.7611111111111111
class1:146/34:class2
Accuracy: 0.8555555555555555
class1:145/35:class2
Accuracy:

In [13]:
def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(10)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = digits.images.reshape(digits.images.shape[0], digits.images.shape[1] * digits.images.shape[2])[j]
    test_y = digits.target[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

In [14]:
pred_value = []
error_list = []

for j in range(1617,1617+180):
    pred = []
    for i in range(511):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(10):
        error.append(511 - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )

In [15]:
list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])

In [16]:
np.sum(list_P == digits.target[1617:])/180

0.8833333333333333

In [17]:
digits.target[1617:]

array([5, 2, 8, 0, 1, 7, 6, 3, 2, 1, 7, 4, 6, 3, 1, 3, 9, 1, 7, 6, 8, 4,
       3, 1, 4, 0, 5, 3, 6, 9, 6, 1, 7, 5, 4, 4, 7, 2, 2, 5, 7, 9, 5, 4,
       4, 9, 0, 8, 9, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5,
       6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 9, 5, 5, 6, 5, 0, 9,
       8, 9, 8, 4, 1, 7, 7, 3, 5, 1, 0, 0, 2, 2, 7, 8, 2, 0, 1, 2, 6, 3,
       3, 7, 3, 3, 4, 6, 6, 6, 4, 9, 1, 5, 0, 9, 5, 2, 8, 2, 0, 0, 1, 7,
       6, 3, 2, 1, 7, 4, 6, 3, 1, 3, 9, 1, 7, 6, 8, 4, 3, 1, 4, 0, 5, 3,
       6, 9, 6, 1, 7, 5, 4, 4, 7, 2, 8, 2, 2, 5, 7, 9, 5, 4, 8, 8, 4, 9,
       0, 8, 9, 8])

In [18]:
# list_P = []
# for i in pred_value:
#     print(i[0])

In [19]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(list_P, digits.target[1617:])
print(cm)

[[16  0  0  0  0  0  0  0  0  0]
 [ 0 18  0  2  0  0  0  0  2  0]
 [ 0  0 17  0  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  2  1]
 [ 0  1  0  0 18  0  0  1  0  0]
 [ 0  0  0  2  0 18  0  0  0  2]
 [ 0  0  0  1  0  0 18  0  0  0]
 [ 0  0  0  2  0  0  0 18  1  1]
 [ 0  0  0  1  2  0  0  0 12  0]
 [ 0  0  0  0  0  0  0  0  0 14]]


In [20]:
import seaborn as sn


df_cm = pd.DataFrame(cm, index = [i for i in "0123456789"],
                  columns = [i for i in "0123456789"])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)
plt.xlabel('Real Label')
plt.ylabel('Prediction Label')
plt.show()

In [21]:
plt.hist(list_P)

(array([16., 22., 17., 13., 20., 22., 19., 22., 15., 14.]),
 array([0. , 0.9, 1.8, 2.7, 3.6, 4.5, 5.4, 6.3, 7.2, 8.1, 9. ]),
 <a list of 10 Patch objects>)

In [22]:
plt.hist(digits.target[1617:])

(array([16., 19., 17., 18., 20., 18., 18., 19., 17., 18.]),
 array([0. , 0.9, 1.8, 2.7, 3.6, 4.5, 5.4, 6.3, 7.2, 8.1, 9. ]),
 <a list of 10 Patch objects>)

In [23]:
list_P = []
for i,p_value in enumerate(pred_value):
    print("#{} min_error:{} index:{} real:{}".format(i, error_list[i], p_value[0], digits.target[1617+i]))


#0 min_error:132 index:[5] real:5
#1 min_error:151 index:[2] real:2
#2 min_error:161 index:[1] real:8
#3 min_error:40 index:[0] real:0
#4 min_error:120 index:[1] real:1
#5 min_error:111 index:[7] real:7
#6 min_error:63 index:[6] real:6
#7 min_error:116 index:[3] real:3
#8 min_error:78 index:[2] real:2
#9 min_error:123 index:[1] real:1
#10 min_error:154 index:[7] real:7
#11 min_error:173 index:[8] real:4
#12 min_error:84 index:[6] real:6
#13 min_error:118 index:[3] real:3
#14 min_error:115 index:[1] real:1
#15 min_error:151 index:[3] real:3
#16 min_error:158 index:[7] real:9
#17 min_error:108 index:[1] real:1
#18 min_error:159 index:[7] real:7
#19 min_error:55 index:[6] real:6
#20 min_error:157 index:[8] real:8
#21 min_error:90 index:[4] real:4
#22 min_error:113 index:[3] real:3
#23 min_error:128 index:[1] real:1
#24 min_error:87 index:[4] real:4
#25 min_error:66 index:[0] real:0
#26 min_error:98 index:[5] real:5
#27 min_error:101 index:[3] real:3
#28 min_error:142 index:[6] real:6
#29 

In [24]:
# program to check if there is exist a path between two vertices 
# of a graph 

from collections import defaultdict 

#This class represents a directed graph using adjacency list representation 
class Graph: 

	def __init__(self,vertices): 
		self.V= vertices #No. of vertices 
		self.graph = defaultdict(list) # default dictionary to store graph 

	# function to add an edge to graph 
	def addEdge(self,u,v): 
		self.graph[u].append(v) 
	
	# Use BFS to check path between s and d 
	def isReachable(self, s, d): 
		# Mark all the vertices as not visited 
		visited =[False]*(self.V) 

		# Create a queue for BFS 
		queue=[] 

		# Mark the source node as visited and enqueue it 
		queue.append(s) 
		visited[s] = True

		while queue: 

			#Dequeue a vertex from queue 
			n = queue.pop(0) 
			
			# If this adjacent node is the destination node, 
			# then return true 
			if n == d: 
				return True

			# Else, continue to do BFS 
			for i in self.graph[n]: 
				if visited[i] == False: 
					queue.append(i) 
					visited[i] = True
		# If BFS is complete without visited d 
		return False
    
    
def findAllPath(graph,start,end,path=[]):
    path = path +[start]
    if start == end:
        return [path]
 
    paths = [] #存储所有路径    
    for node in graph[start]:
        if node not in path:
            newpaths = findAllPath(graph,node,end,path) 
            for newpath in newpaths:
                paths.append(newpath)
    return paths

numbers = [*range(2)]
number_of_classification = len(numbers)

list_nodes_number = []
list_connection_number = []

for winner in winner_list:
    used_nodes = list(winner.nodes.keys())
    # create a dict for mapping 
    l1 = numbers + config.genome_config.input_keys + used_nodes[number_of_classification:]
    l2 = range(len(l1))
    dict_nodes = dict(zip(l1, l2))

    ### 将节点加入图
    g = Graph(len(l1))# inputs + outputs + used  
    # add connections in the graph
    for cg in winner.connections.values():
        if cg.enabled:
            g.addEdge(dict_nodes[cg.key[0]],dict_nodes[cg.key[1]])

    v = numbers
    list_connections = []
    for vi in v:
        for u in range(number_of_classification, len(config.genome_config.input_keys) + number_of_classification):
            list_connections.append(g.isReachable(u, vi))

    nodes_inputs = range(number_of_classification, number_of_classification+ 64)
    nodes_outputs = range(number_of_classification)
    nodes_mid = range(number_of_classification + 64, len(dict_nodes))

    # 找到所有输入和输出连接的路
    all_path = []
    for u in nodes_inputs:
        for v in nodes_outputs:
            path = findAllPath(g.graph, u, v)
            if path:
                all_path = all_path + path

    # 得到最长路，以及每个节点在哪个层级
    max_length = max([len(x) for x in all_path])
    nodes_tuples_list = []
    for path in all_path:
        #print(path)
        for node in path:
            nodes_tuples_list.append([node, path.index(node)])

    # 确定节点的层级（消除重复）       
    nodes_tuples_fixed_list = []
    for index in range(max_length):
        for nodes in nodes_tuples_list:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                if nodes not in nodes_tuples_fixed_list:
                    nodes_tuples_fixed_list.append(nodes)


    # 所有有连接的节点
    all_used_nodes = []
    for node in nodes_tuples_fixed_list:
        if node[0] not in all_used_nodes:
            all_used_nodes.append(node[0])

    # 确定节点的层级（最终确定）
    nodes_tuples_fixed_list_final = []

    for node in all_used_nodes:
        if node in nodes_inputs:
            nodes_tuples_fixed_list_final.append([node, 0])
        elif node in nodes_outputs:
            nodes_tuples_fixed_list_final.append([node, max_length - 1])
        else:
            list_node = []
            for nodes in nodes_tuples_fixed_list:
                if node == nodes[0]:
                    list_node.append(nodes)
            layer = 0
            for ele in list_node:
                # 删除重复项，取最大
                if ele[1] > layer:
                    layer = ele[1]
            nodes_tuples_fixed_list_final.append([node,layer])


    # 每个层级的节点合数
    length_of_layers = []
    number_of_nodes = 0
    for index in range(max_length):
        for nodes in nodes_tuples_fixed_list_final:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                number_of_nodes += 1
        length_of_layers.append(number_of_nodes)

    length_of_each_layer = []
    length_of_each_layer.append(length_of_layers[0])
    for i in range(1, max_length):
        length_of_each_layer.append(length_of_layers[i] - length_of_layers[i-1])

    # 输出每个层级的节点个数
    #print("length of each layers:", length_of_each_layer)

    # 所有端到端的路
    all_path_side2side = []
    for path in all_path:
        if len(path) == 2:
            all_path_side2side.append(path)
        else:
            for i in range(len(path)-1):
                all_path_side2side.append([path[i],path[i+1]])

    # 定义节点到绘图的字典
    dict_nodes_graph = dict()
    count_number_layer = [0] * max_length
    for nodes in nodes_tuples_fixed_list_final:
        node, layer = nodes
        dict_nodes_graph[node] = [count_number_layer[layer], layer]
        count_number_layer[layer] += 1
    list_nodes_number.append(np.sum(length_of_each_layer))
    list_connection_number.append(len(all_path_side2side))
    #print("Number of nodes:{} Number of connections:{}".format(np.sum(length_of_each_layer),len(all_path_side2side)))
print("Total nodes:{} Total connections::{}".format(np.sum(list_nodes_number), np.sum(list_connection_number)))

Total nodes:8666 Total connections::8571


In [26]:
necoc = 511
pred_value = []
error_list = []

for j in range(1617):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(10):
        error.append(10 - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )

list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
print("Testset acc:{}".format(np.sum(list_P == digits.target[:1617])/1617))
winner_fitness = []
for winner in winner_list:
    #print(winner.fitness)
    winner_fitness.append(winner.fitness)
print("Avg Base Learner:{}".format(np.mean(winner_fitness)))

Testset acc:0.9010513296227582
Avg Base Learner:0.7827014100427576
